In [1]:
import numpy as np
import json
from scipy.optimize import minimize

In [2]:
with open('data.json', 'r', encoding='utf-8') as f:
    raw_data = json.load(f)

data = {}
scale = 10 # q-value가 너무 크게 변해 alpha가 0에 가까워지는 현상 방지

for subj, lists in raw_data.items():
    choices, gains, losses, rewards = lists
    choices = (np.array(choices, dtype=int) - 1) # 1~4 -> 0~3
    gains = np.array(gains, dtype=float) / scale
    losses = np.array(losses, dtype=float) / scale
    rewards = np.array(rewards, dtype=float) / scale

    data[int(subj)] = {
        "choices": choices,
        "gains": gains,
        "losses": losses,
        "rewards": rewards,
    }

In [3]:
#MLE 시행
def neg_log_likelihood(params, trials, vs = False):
  if vs: # vs: valence-specific Q-learning
    a_p, a_m, b, w = params # alpha_plus, alpha_minus, beta, weight
    Q_pos = np.zeros(4)
    Q_neg = np.zeros(4)
  else:
    a, b = params # alpha, beta
    Q = np.zeros(4) # initialize Q-values for 4 decks (A, B, C, D)

  nll = 0

  for t in trials:
    if vs:
      action, gain, loss = t
      Q_comb = w * Q_pos - (1 - w) * Q_neg

      logits = b * Q_comb
      logits -= np.max(logits) # overflow 방지
      probs = np.exp(logits)
      probs /= np.sum(probs)
      nll -= np.log(probs[action] + 1e-12)

      Q_pos[action] += a_p * (gain - Q_pos[action])
      if loss != 0:
        Q_neg[action] += a_m * (abs(loss) - Q_neg[action])

    else:
      action, reward = t

      logits = b * Q
      logits -= np.max(logits)
      probs = np.exp(logits)
      probs /= np.sum(probs)
      nll -= np.log(probs[action] + 1e-12)

      Q[action] += a * (reward - Q[action])

  return nll

In [4]:
params_sub = [] ## 각 참가자의 추정 파라미터를 저장할 리스트
init_q = [0.0001, 1.0] # q learning의 초기 alpha, beta
bounds_q = [(0, 1), (0, 1000)] ## alpha, beta bound
init_vs = [0.0001, 0.0001, 1.0, 0.5] # VSQ learning의 초기 a_p, a_m, b, w, weight
bounds_vs = [(0, 1), (0, 1), (0, 1000), (0, 1)] ## bound

## 파라미터 추정하기 (전체 subject)
for subj, d in data.items():
    ## Q leanring 학습 데이터 준비
    trials_q  = list(zip(d["choices"], d["rewards"]))

    ## VSQ 학습 데이터 준비
    trials_vs = list(zip(d["choices"], d["gains"], d["losses"]))

    params_q = minimize(
        neg_log_likelihood, ## 최적화 대상 함수
        x0 = init_q, ## 초기값
        args = (trials_q, False), ## 인자
        bounds = bounds_q ## 파라미터 제한 범위
    )

    params_vs = minimize(
        neg_log_likelihood, ## 최적화 대상 함수
        x0 = init_vs, ## 초기값
        args = (trials_vs, True), #인자
        bounds = bounds_vs #파라미터 제한 범위
    )
    ## 결과 저장
    params_sub.append({
        "subj": subj,
        "q_alpha": params_q.x[0],
        "q_beta": params_q.x[1],
        "vs_alpha_plus": params_vs.x[0],
        "vs_alpha_minus": params_vs.x[1],
        "vs_beta": params_vs.x[2],
        "vs_w": params_vs.x[3],
        "nll_q": params_q.fun,
        "nll_vs": params_vs.fun
    })
    print(f"Subject {subj}: nll_q={params_q.fun:.2f}, nll_vs={params_vs.fun:.2f}")
    print(f"  Q-learning params:  α={params_q.x[0]:.10f}, β={params_q.x[1]:.10f}")
    print(f"  VSQ-learning params: α+={params_vs.x[0]:.10f}, α-={params_vs.x[1]:.10f}, β={params_vs.x[2]:.10f}, w={params_vs.x[3]:.3f}\n")

with open("params_sub.json", "w", encoding="utf-8") as f:
  json.dump(params_sub, f, ensure_ascii=False, indent=4)

Subject 1: nll_q=138.63, nll_vs=130.83
  Q-learning params:  α=0.0000000000, β=0.9129784095
  VSQ-learning params: α+=0.0053393251, α-=0.0002989360, β=24.3813227373, w=0.124

Subject 2: nll_q=138.35, nll_vs=134.35
  Q-learning params:  α=0.0015597628, β=1.0000498137
  VSQ-learning params: α+=0.0000678940, α-=0.0002301069, β=234.6989309187, w=0.875

Subject 3: nll_q=138.63, nll_vs=127.77
  Q-learning params:  α=0.0000000000, β=0.9129698837
  VSQ-learning params: α+=0.0046659174, α-=0.0015402761, β=7.7146258918, w=0.469

Subject 4: nll_q=136.60, nll_vs=136.47
  Q-learning params:  α=0.0050581547, β=1.0004141489
  VSQ-learning params: α+=0.0120165838, α-=0.0002291010, β=26.5882587314, w=0.028

Subject 5: nll_q=137.06, nll_vs=136.84
  Q-learning params:  α=0.0040363075, β=1.0003114656
  VSQ-learning params: α+=0.0265546264, α-=0.0003503119, β=15.2693247073, w=0.024

Subject 6: nll_q=138.63, nll_vs=81.84
  Q-learning params:  α=0.0000000000, β=0.4839981802
  VSQ-learning params: α+=0.066599

In [5]:
#학습한 learners만 추출
import statsmodels.api as sm

good_decks = {2, 3}
learners = set()
params_sub_1 = []

for subj, d in data.items():
    choices = d["choices"]
    y = np.isin(choices, list(good_decks)).astype(int)
    trials = np.arange(len(choices))
    trials = (trials - np.mean(trials)) / np.std(trials)
    X = sm.add_constant(trials)

    try:
        model = sm.Logit(y, X)
        result = model.fit(disp=False, method='lbfgs', maxiter=200)
        slope = result.params[1]
        pvalue = result.pvalues[1]
        if slope > 0 and pvalue < 0.05:
            learners.add(subj)
    except:
        continue

print(f"전체 {len(data)}명 중 학습자 수: {len(learners)}명")

## 파라미터 추정하기 (learners)
for subj in learners:
    d = data[subj]

    trials_q  = list(zip(d["choices"], d["rewards"]))

    ## VSQ 학습 데이터 준비
    trials_vs = list(zip(d["choices"], d["gains"], d["losses"]))

    params_q = minimize(
        neg_log_likelihood, ## 최적화 대상 함수
        x0 = init_q, ## 초기값
        args = (trials_q, False), ## 인자
        bounds = bounds_q ## 파라미터 제한 범위
    )

    params_vs = minimize(
        neg_log_likelihood, ## 최적화 대상 함수
        x0 = init_vs, ## 초기값
        args = (trials_vs, True), #인자
        bounds = bounds_vs #파라미터 제한 범위
    )
    ## 결과 저장
    params_sub_1.append({
        "subj": subj,
        "q_alpha": params_q.x[0],
        "q_beta": params_q.x[1],
        "vs_alpha_plus": params_vs.x[0],
        "vs_alpha_minus": params_vs.x[1],
        "vs_beta": params_vs.x[2],
        "vs_w": params_vs.x[3],
        "nll_q": params_q.fun,
        "nll_vs": params_vs.fun
    })
    print(f"Subject {subj}: nll_q={params_q.fun:.2f}, nll_vs={params_vs.fun:.2f}")
    print(f"  Q-learning params:  α={params_q.x[0]:.10f}, β={params_q.x[1]:.10f}")
    print(f"  VSQ-learning params: α+={params_vs.x[0]:.10f}, α-={params_vs.x[1]:.10f}, β={params_vs.x[2]:.10f}, w={params_vs.x[3]:.3f}\n")

전체 602명 중 학습자 수: 198명
Subject 513: nll_q=138.63, nll_vs=125.24
  Q-learning params:  α=0.0000000000, β=0.9222893612
  VSQ-learning params: α+=0.2333746486, α-=0.0000931471, β=28.5172465420, w=0.006

Subject 514: nll_q=36.04, nll_vs=73.13
  Q-learning params:  α=0.3116040753, β=0.6482340881
  VSQ-learning params: α+=0.0000942192, α-=0.0000000000, β=286.2291502982, w=1.000

Subject 515: nll_q=95.75, nll_vs=128.77
  Q-learning params:  α=0.0000942328, β=229.2099478230
  VSQ-learning params: α+=0.0000426073, α-=0.0000000000, β=93.7644005155, w=1.000

Subject 5: nll_q=137.06, nll_vs=136.84
  Q-learning params:  α=0.0040363075, β=1.0003114656
  VSQ-learning params: α+=0.0265546264, α-=0.0003503119, β=15.2693247073, w=0.024

Subject 517: nll_q=124.24, nll_vs=121.91
  Q-learning params:  α=0.0000841173, β=163.2315528845
  VSQ-learning params: α+=0.0000457455, α-=0.0002477995, β=204.0423776165, w=0.774

Subject 519: nll_q=138.63, nll_vs=129.24
  Q-learning params:  α=0.0000000000, β=0.876931155

In [6]:
params_sub_1_sorted = sorted(params_sub_1, key=lambda x: x["subj"])

with open("learners_sub.json", "w", encoding="utf-8") as f:
    json.dump(params_sub_1_sorted, f, ensure_ascii=False, indent=4)

In [7]:
## 추가: 대략적으로 잘 작동하는 파라미터 찾기 위한 grid search
import pandas as pd

alphas = [0, 0.00001, 0.0001, 0.001, 0.01, 0.05, 0.1, 0.3]
betas  = [0, 0.5, 1.0, 3.0, 10.0]

nll_mat = np.zeros((len(alphas), len(betas)))

for subj, d in data.items():
    ## Q leanring 학습 데이터 준비
    trials_q  = list(zip(d["choices"], d["rewards"]))

    for i, alpha in enumerate(alphas):
        for j, beta in enumerate(betas):
            nll = neg_log_likelihood([alpha, beta], trials_q, vs=False)
            nll_mat[i, j] = nll

    df_nll = pd.DataFrame(nll_mat, index=[f"α={a}" for a in alphas],
                                    columns=[f"β={b}" for b in betas])
    print("subj: ", subj)
    print(df_nll)

subj:  1
                 β=0       β=0.5       β=1.0        β=3.0       β=10.0
α=0       138.629436  138.629436  138.629436   138.629436   138.629436
α=1e-05   138.629436  138.633706  138.637981   138.655125   138.715700
α=0.0001  138.629436  138.672269  138.715553   138.893202   139.571510
α=0.001   138.629436  139.070552  139.555972   141.931663   155.195775
α=0.01    138.629436  144.081000  153.023848   210.934662   488.454008
α=0.05    138.629436  173.466586  236.839060   544.750332  1287.178416
α=0.1     138.629436  206.475928  320.045447   809.313105  1485.791805
α=0.3     138.629436  308.253695  536.966766  1064.485073  1573.570754
subj:  2
                 β=0       β=0.5       β=1.0        β=3.0       β=10.0
α=0       138.629436  138.629436  138.629436   138.629436   138.629436
α=1e-05   138.629436  138.627608  138.625786   138.618560   138.594057
α=0.0001  138.629436  138.611427  138.594040   138.530708   138.386753
α=0.001   138.629436  138.476199  138.383920   138.607362  